In [17]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os

In [18]:
os.environ['SPARK_HOME'] = r'C:\spark\spark-3.5.4-bin-hadoop3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [19]:
spark = (
    SparkSession
    .builder
    .appName("PySpark Zero to Hero")
    .master("local[*]")
    .config("spark.executor.memory", "16g")
    .config("spark.driver.memory", "16g")
    .config("spark.executor.cores", "4")
    .config("spark.sql.shuffle.partitions", "80")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", "2")
    .config("spark.dynamicAllocation.initialExecutors", "24")
    .config("spark.dynamicAllocation.maxExecutors", "50")
    .config("spark.shuffle.service.enabled", "true")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

In [20]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

In [21]:
emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [22]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [23]:
emp.schema

StructType([StructField('employee_id', StringType(), True), StructField('department_id', StringType(), True), StructField('name', StringType(), True), StructField('age', StringType(), True), StructField('gender', StringType(), True), StructField('salary', StringType(), True), StructField('hire_date', StringType(), True)])

In [24]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

schema_string = 'name string age int'

schema_spark = StructType([
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True),
])

In [25]:
emp.salary
emp['salary']

Column<'salary'>

In [26]:
emp_filter = emp.select(F.col('employee_id'), F.expr('name'), emp.age, emp.salary)

In [30]:
emp_filter.show(5)

+-----------+----------+---+------+
|employee_id|      name|age|salary|
+-----------+----------+---+------+
|        001|  John Doe| 30| 50000|
|        002|Jane Smith| 25| 45000|
|        003| Bob Brown| 35| 55000|
|        004| Alice Lee| 28| 48000|
|        005| Jack Chan| 40| 60000|
+-----------+----------+---+------+
only showing top 5 rows



In [28]:
emp_casted = emp_filter.select(
    F.expr('employee_id as emp_id'),
    emp.name,
    F.expr('cast(age as int) as age'),
    emp.salary
)

In [29]:
emp_casted.show(5)

+------+----------+---+------+
|emp_id|      name|age|salary|
+------+----------+---+------+
|   001|  John Doe| 30| 50000|
|   002|Jane Smith| 25| 45000|
|   003| Bob Brown| 35| 55000|
|   004| Alice Lee| 28| 48000|
|   005| Jack Chan| 40| 60000|
+------+----------+---+------+
only showing top 5 rows



In [31]:
emp_casted_1 = emp_filter.selectExpr(
    'employee_id as emp_id',
    'name',
    'cast(age as int) as age',
    'salary'
)

In [32]:
emp_casted_1.show(5)

+------+----------+---+------+
|emp_id|      name|age|salary|
+------+----------+---+------+
|   001|  John Doe| 30| 50000|
|   002|Jane Smith| 25| 45000|
|   003| Bob Brown| 35| 55000|
|   004| Alice Lee| 28| 48000|
|   005| Jack Chan| 40| 60000|
+------+----------+---+------+
only showing top 5 rows



In [35]:
emp_final = emp_casted.select(
    'emp_id', 'name', 'age', 'salary'
).where(
    F.col('age') > 30
)

In [37]:
emp_final.show(5)

+------+-------------+---+------+
|emp_id|         name|age|salary|
+------+-------------+---+------+
|   003|    Bob Brown| 35| 55000|
|   005|    Jack Chan| 40| 60000|
|   006|    Jill Wong| 32| 52000|
|   007|James Johnson| 42| 70000|
|   009|      Tom Tan| 33| 58000|
+------+-------------+---+------+
only showing top 5 rows



In [39]:
schema_str = 'name string, age int'

from pyspark.sql.types import _parse_datatype_string

schema_spark = _parse_datatype_string(schema_str)

schema_spark

StructType([StructField('name', StringType(), True), StructField('age', IntegerType(), True)])

# https://www.youtube.com/watch?v=EiPCK2y_QPk&list=PL2IsFZBGM_IHCl9zhRVC1EXTomkEp_1zm&index=7